In [1]:
# --- CELLULE 1 : IMPORTATION ---
import pandas as pd
import numpy as np
print("Librairies chargées.")

# --- CELLULE 2 : CHARGEMENT ---
df = pd.read_csv('../DATA/RAW/dirty_cafe_sales.csv')

# --- CELLULE 3 : STANDARDISATION ---
# On remplace les textes parasites par NaN (Vide)
df.replace(['ERROR', 'UNKNOWN'], np.nan, inplace=True)
print("Standardisation terminée : 'ERROR' et 'UNKNOWN' remplacés par NaN.")

# --- CELLULE 4 : STRATÉGIE 1 - IMPUTATION CATÉGORIELLE ---
# Pour Location et Payment, on met "Unknown" pour ne pas perdre la ligne
cols_cat = ['Location', 'Payment Method']
for col in cols_cat:
    df[col] = df[col].fillna('Unknown')

# --- CELLULE 5 : CONVERSION DES TYPES ---
# Maintenant que c'est propre, on force le typage numérique
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

# --- CELLULE 6 : STRATÉGIE 2 - LOGIQUE MÉTIER (Réparation Item/Prix) ---
# Dictionnaires de référence
mapping_prix = {1.0: 'Cookie', 1.5: 'Tea', 2.0: 'Coffee', 5.0: 'Salad'}
mapping_item = {'Cookie': 1.0, 'Tea': 1.5, 'Coffee': 2.0, 'Salad': 5.0, 
                'Cake': 3.0, 'Sandwich': 4.0, 'Smoothie': 4.0, 'Juice': 3.0}

# A. Si Item manque, on devine grâce au Prix
df['Item'] = df['Item'].fillna(df['Price Per Unit'].map(mapping_prix))

# B. Si Prix manque, on devine grâce à l'Item
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Item'].map(mapping_item))

# --- CELLULE 7 : STRATÉGIE 3 - CONSOLIDATION MATHÉMATIQUE ---
# A. Qty = Total / Prix (si Qty manque)
mask_qty = df['Quantity'].isna() & df['Total Spent'].notna() & df['Price Per Unit'].notna()
df.loc[mask_qty, 'Quantity'] = df.loc[mask_qty, 'Total Spent'] / df.loc[mask_qty, 'Price Per Unit']

# B. Total = Qty * Prix (Recalcul forcé pour corriger les erreurs de calcul)
df['Total Spent'] = df['Quantity'] * df['Price Per Unit']

# --- CELLULE 8 : STRATÉGIE 4 - TEMPORELLE ---
# Forward Fill pour les dates
df['Transaction Date'] = df['Transaction Date'].ffill()

# --- CELLULE 9 : NETTOYAGE FINAL ---
# 1. Items toujours inconnus -> "Unknown Item"
df['Item'] = df['Item'].fillna('Unknown Item')

# 2. Suppression des lignes vides irrécupérables (sans prix ni total)
df_final = df.dropna(subset=['Quantity', 'Price Per Unit', 'Total Spent'])

# --- CELLULE 10 : VÉRIFICATION ET SAUVEGARDE ---
print("--- Check Final (Doit afficher 0 partout) ---")
print(df_final.isnull().sum())

df_final.to_csv('../DATA/PROCESSED/cleaned_cafe_sales.csv', index=False)
print(f"Dataset propre sauvegardé : {len(df_final)} lignes conservées sur 10 000.")

Librairies chargées.
Standardisation terminée : 'ERROR' et 'UNKNOWN' remplacés par NaN.
--- Check Final (Doit afficher 0 partout) ---
Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64
Dataset propre sauvegardé : 9926 lignes conservées sur 10 000.
